<a href="https://colab.research.google.com/github/crisperchimp/crisperchimp/blob/main/Python_DA_Week3Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#json example

{
    "name": "John Doe",
    "age": 30,
    "city": "New York"
}

{'name': 'John Doe', 'age': 30, 'city': 'New York'}

In [ ]:
import requests
import pandas as pd

In [ ]:
url = "http://api.open-notify.org/iss-now.json"
response = requests.get(url)

In [ ]:
# Fetching data from a public API (this is the current location of the international space station)

if response.status_code == 200:
    data = response.json()

    timestamp = data['timestamp']
    latitude = data['iss_position']['latitude']
    longitude = data['iss_position']['longitude']

    df = pd.DataFrame({
        'timestamp': [timestamp],
        'latitude': [latitude],
        'longitude': [longitude]
    })

    print(df)
else:
    print(f"Error: {response.status_code}")

    timestamp  latitude longitude
0  1726788435  -51.5371  129.0384


In [ ]:
#storing data as a CSV

df.to_csv('iss_location.csv', index=False)

In [ ]:
#storing data in SQLite

import sqlite3

conn = sqlite3.connect('iss_data.db')
df.to_sql('iss_locations', conn, if_exists='append', index=False)
conn.close()

In [ ]:
from datetime import datetime

def fetch_iss_location():
    url = "http://api.open-notify.org/iss-now.json"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()

        timestamp = data['timestamp']
        latitude = data['iss_position']['latitude']
        longitude = data['iss_position']['longitude']

        return {
            'timestamp': datetime.fromtimestamp(data['timestamp']),
            'latitude': float(data['iss_position']['latitude']),
            'longitude': float(data['iss_position']['longitude'])
        }

        print(df)
    else:
        print(f"Error: {response.status_code}")

In [ ]:
iss_data = fetch_iss_location()

In [ ]:
print(iss_data)

{'timestamp': datetime.datetime(2024, 9, 19, 18, 42, 58), 'latitude': -20.3961, 'longitude': -166.4303}


In [ ]:
if iss_data:
    df = pd.DataFrame([iss_data])

    df.to_csv('iss_location.csv', index=False)
    print("Data saved to csv")

    conn = sqlite3.connect('iss_data.db')
    df.to_sql('iss_locations', conn, if_exists='append', index=False)
    conn.close()
    print("data saved to sqlite")
else:
    print("failed to fetch iss location data")

Data saved to csv
data saved to sqlite


In [ ]:
conn = sqlite3.connect('iss_data.db')
db_data = pd.read_sql_query("SELECT * FROM iss_locations", conn)
conn.close()

In [ ]:
db_data

,timestamp,latitude,longitude
0,1726788435,-51.5371,129.0384
1,1726788435,-51.5371,129.0384
2,2024-09-19 18:42:58,-20.3961,-166.4303


In [ ]:
combined_data = pd.concat([db_data, iss_data], ignore_index=True)

TypeError: cannot concatenate object of type '<class 'dict'>'; only Series and DataFrame objs are valid

In [ ]:
import matplotlib.pyplot as plt

# Note: You need to sign up for a free API key at openweathermap.org
API_KEY = ""
CITY = "Chicago"

# Make API request
url = f"http://api.openweathermap.org/data/2.5/forecast?q={CITY}&appid={API_KEY}&units=imperial"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()

    # Extract relevant information
    forecast = []
    for item in data['list']:
        forecast.append({
            'datetime': item['dt_txt'],
            'temperature': item['main']['temp'],
            'humidity': item['main']['humidity']
        })

    # Create DataFrame
    df = pd.DataFrame(forecast)

    # Save to CSV
    df.to_csv(f"{CITY}_weather_forecast.csv", index=False)
    print(f"Data saved to {CITY}_weather_forecast.csv")

    # Simple analysis
    print("\nAverage temperature:", df['temperature'].mean())
    print("Max temperature:", df['temperature'].max())
    print("Min temperature:", df['temperature'].min())

    # Simple visualization
    plt.figure(figsize=(10, 6))
    plt.plot(df['datetime'], df['temperature'])
    plt.title(f"Temperature Forecast for {CITY}")
    plt.xlabel("Date")
    plt.ylabel("Temperature (°F)")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print(f"Error: {response.status_code}")

Error: 401
